In [246]:
import pandas as pd
import numpy as np

In [247]:
test = pd.read_csv('Task/test.csv')
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [248]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  1614 non-null   object
dtypes: object(1)
memory usage: 12.7+ KB


In [249]:
test.sample(3)

,Smiles
42,Cc1cc(C(=O)Nc2cccc(C(=O)Nc3cccc(C(F)(F)F)c3)c2...
64,NC(=O)c1cnc(N[C@@H]2CCCC[C@@H]2N)nc1Nc1cccc(-n...
1050,N[C@@H](CC(=O)NO)C(=O)O


In [250]:
train = pd.read_csv('Task/train.csv')
train.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [251]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5557 entries, 0 to 5556
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Smiles  5557 non-null   object
 1   Active  5557 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 49.0+ KB


In [252]:
train.sample(3)

,Smiles,Active
3930,CCS(=O)(=O)N(Cc1cccnc1)c1cccc(-c2ccc(C#N)cc2)c1,False
1458,Cc1ncc([N+](=O)[O-])n1C,False
312,CCN(CC)Cc1ccc2cc(COC(=O)Nc3ccc(C(=O)NO)cc3)ccc2c1,False


In [253]:
list(train['Active'].unique())

[False, True]

In [254]:
list_active = {False: 0, True: 1}

In [255]:
train['Active'] = train['Active'].map(list_active)

In [256]:
train.sample()

,Smiles,Active
4701,COCCNC(=O)Cn1cnc([N+](=O)[O-])n1,0


In [257]:
train['train'] = 1
test['train'] = 0

In [258]:
df = pd.concat([train, test], ignore_index=True)

In [259]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7171 entries, 0 to 7170
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Smiles  7171 non-null   object 
 1   Active  5557 non-null   float64
 2   train   7171 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 168.2+ KB


In [260]:
df['train'].unique()

array([1, 0], dtype=int64)

In [261]:
df[df['train'] == 1]['Active'].value_counts()

0.0    5351
1.0     206
Name: Active, dtype: int64

In [262]:
from collections import Counter

atoms = Counter()
for i in df['Smiles'].unique():
    for j in i:
        atoms[j] += 1

In [263]:
atoms['C']

67272

In [264]:
new_columnes = list(atoms.keys())

In [265]:
print(new_columnes)

['C', 'O', 'c', '1', '2', '[', 'n', 'H', ']', '(', 'N', ')', '@', '.', 'l', '=', 'S', '3', '4', 'F', '-', 's', '/', '5', 'o', 'a', '+', '#', 'I', 'P', 'B', 'r', '\\', 'Z', '6', '7', '8', 'e', 'A', 'K', 'M', 'g', 'i', 'L', '9']


In [266]:
df['len_smiles'] = df['Smiles'].apply(lambda x: len(x))

In [267]:
df['len_smiles'].max()

707

In [268]:
df['len_smiles'].describe()

count    7171.000000
mean       51.122438
std        31.876511
min         6.000000
25%        34.000000
50%        46.000000
75%        60.000000
max       707.000000
Name: len_smiles, dtype: float64

In [269]:
df[df['len_smiles'] > 60]['Smiles'].count()

1732

In [270]:
df['smiles_list'] = df['Smiles'].apply(lambda x: list(x))

In [271]:
df['smiles_list'] = df['smiles_list'].apply(lambda x: count_i(x))

In [272]:
for i in new_columnes:
    df[i] = df['smiles_list'].apply(lambda x: x[i])

In [273]:
def count_i(x):
    atoms = Counter()
    for i in x:
        atoms[i] += 1
    return atoms

In [274]:
df.sample(3)

,Smiles,Active,train,len_smiles,smiles_list,C,O,c,1,2,...,7,8,e,A,K,M,g,i,L,9
365,CCOC(=O)C1=C(COCCN)NC(C)=C(C(=O)OC)C1c1ccccc1C...,0.0,1,66,"{'C': 15, 'O': 8, '(': 7, '=': 6, ')': 7, '1':...",15,8,12,6,0,...,0,0,0,0,0,0,0,0,0,0
6353,Nc1ccc(-c2ccc3ccccc3n2)cc1,NaN,0,26,"{'N': 1, 'c': 15, '1': 2, '(': 1, '-': 1, '2':...",0,0,15,2,2,...,0,0,0,0,0,0,0,0,0,0
4,Fc1ccccc1CNCc1ccc(-c2ccnc3[nH]ccc23)cc1,0.0,1,39,"{'F': 1, 'c': 19, '1': 4, 'C': 2, 'N': 1, '(':...",2,0,19,4,2,...,0,0,0,0,0,0,0,0,0,0


In [275]:
df.sample(3)

,Smiles,Active,train,len_smiles,smiles_list,C,O,c,1,2,...,7,8,e,A,K,M,g,i,L,9
3175,CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1[C@H...,0.0,1,74,"{'C': 18, 'O': 5, '(': 6, '=': 3, ')': 6, '[':...",18,5,6,4,2,...,0,0,0,0,0,0,0,0,0,0
4376,CN[C@@H](C)C(=O)N[C@H](C(=O)N1CCC[C@H]1c1nc(C(...,0.0,1,73,"{'C': 17, 'N': 3, '[': 3, '@': 4, 'H': 3, ']':...",17,3,9,6,2,...,0,0,0,0,0,0,0,0,0,0
3457,C[C@]1(Cn2ccnn2)[C@H](C(=O)O)N2C(=O)C[C@H]2S1(...,0.0,1,51,"{'C': 8, '[': 3, '@': 3, ']': 3, '1': 2, '(': ...",8,5,2,2,4,...,0,0,0,0,0,0,0,0,0,0


In [276]:
df.drop(['smiles_list'], axis = 1, inplace = True)

In [277]:
df.set_index(['Smiles'], inplace = True)

In [278]:
df.sample(3)

,Active,train,len_smiles,C,O,c,1,2,[,n,...,7,8,e,A,K,M,g,i,L,9
Smiles,,,,,,,,,,,,,,,,,,,,,
O=C1/C(=C/c2ccccc2)C[C@@]2(O)C3Cc4ccc(O)c5c4[C@@]2(CCN3CC2CC2)[C@H]1O5,0.0,1,70,15,4,12,2,6,3,0,...,0,0,0,0,0,0,0,0,0,0
Cc1cc(-c2ccc(/N=N/c3ccc4c(S(=O)(=O)[O-])cc(S(=O)(=O)[O-])c(N)c4c3O)c(C)c2)ccc1/N=N/c1ccc2c(S(=O)(=O)[O-])cc(S(=O)(=O)[O-])c(N)c2c1O.[Na+].[Na+].[Na+].[Na+],1.0,1,155,2,14,32,4,4,8,0,...,0,0,0,0,0,0,0,0,0,0
CC1=C(C)C(=O)C([C@@H](CCCCCC(=O)O)c2ccccc2)=C(C)C1=O,0.0,1,52,16,4,6,2,2,1,0,...,0,0,0,0,0,0,0,0,0,0


In [279]:
train_new = df[df['train'] == 1]
test_new = df[df['train'] == 0]

In [280]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5557 entries, COc1ccc2[nH]cc(CCN)c2c1 to CCCCNc1ccc(C(=O)OCCN(C)C)cc1.Cl
Data columns (total 48 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Active      5557 non-null   float64
 1   train       5557 non-null   int64  
 2   len_smiles  5557 non-null   int64  
 3   C           5557 non-null   int64  
 4   O           5557 non-null   int64  
 5   c           5557 non-null   int64  
 6   1           5557 non-null   int64  
 7   2           5557 non-null   int64  
 8   [           5557 non-null   int64  
 9   n           5557 non-null   int64  
 10  H           5557 non-null   int64  
 11  ]           5557 non-null   int64  
 12  (           5557 non-null   int64  
 13  N           5557 non-null   int64  
 14  )           5557 non-null   int64  
 15  @           5557 non-null   int64  
 16  .           5557 non-null   int64  
 17  l           5557 non-null   int64  
 18  =           5557 non-nul

In [281]:
X = train_new.drop(['Active'], axis = 1)
y = train_new['Active']

In [283]:
from sklearn import model_selection, datasets, metrics, tree 

In [284]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, 
                                                                                    test_size=0.30, random_state=42)

In [285]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

In [302]:
clf = tree.DecisionTreeClassifier()
clf.fit(train_data, train_labels)

DecisionTreeClassifier()

In [303]:
predictions_clf = clf.predict(test_data)
metrics.f1_score(test_labels, predictions_clf)

0.1605839416058394

In [297]:
bagging_trees = BaggingClassifier(clf, n_estimators = 1500)
bagging_trees.fit(train_data,train_labels)
predictions_bg = bagging_trees.predict(test_data)

In [298]:
metrics.f1_score(test_labels, predictions_bg)

0.027027027027027025

In [299]:
from lazypredict.Supervised import LazyClassifier

C:\Users\Rusla\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [300]:
clf_lc = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf_lc.fit(train_data, test_data, train_labels, test_labels)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:19<00:00,  1.48it/s]


In [301]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
DecisionTreeClassifier             0.93               0.55     0.55      0.93   
NearestCentroid                    0.75               0.54     0.54      0.83   
ExtraTreeClassifier                0.93               0.53     0.53      0.93   
LabelSpreading                     0.94               0.52     0.52      0.93   
LabelPropagation                   0.94               0.52     0.52      0.93   
XGBClassifier                      0.96               0.52     0.52      0.94   
Perceptron                         0.94               0.51     0.51      0.93   
BaggingClassifier                  0.95               0.51     0.51      0.94   
SGDClassifier                      0.95               0.51     0.51      0.94   
LinearDiscriminantAnalysis         0.95               0.51     0.51      0.94   
QuadraticDiscriminantAnalysi